<div>
<img src="opencosmo_simple_transparent.png" width="600">
</div>

# OpenCosmo Tutorial - lightcone data



<div class="alert alert-block alert-info"><b>Note: </b> This tutorial is a little different, we're updating last minute and I don't have any examples on hand so you're going to try this out with me and work it out together! I'll give code snippets and my best guesses but you'll need to put together the details. This is why it's an advanced tutorial :)  </div> 


## The portal

First navigate to Halo LC Query  and select something like the following:


|Complete          |  Option |
|:------------------|:--------|
| Simulation               | Frontier GO | 
| Limit     | 1000      | 
| Lower step              | 500      | 
| Upper step               | 624      | 
| M200c high | 1e16      | 
| M200c low| 1e14      | 
| theta high | 0.5      | 
| theta low| 0.1     | 
| phi high | 0.5      | 
| phi low| 0.1      | 

And if we're very lucky you'll get some halos! These are the standard mathematical theta, phi bounds on the lightcone in radians, as well as redshift slicing through the step number (500 is roughly z=0.15) and a mass cut. 

Then navigate to the CMB map query, and you can get hold of the CMB lensing map (with contributions up to z=5) from the Frontier E simulation to go along with it 


|Complete          |  Option |
|:------------------|:--------|
| Simulation               | Frontier GO | 
| nside out | 2048      | 


If you want you can get a small patch with a theta,phi position and a radius, and it'll give you an image and the map in a healsparse format. However this set-up will downgrade the map to a reasonable resolution for you so you should be able to manage the whole thing. 


<div class="alert alert-block alert-info"><b>Note: </b> Some of the options are changing quickly, and they may sometimes break! If you are unsure about anything just ask!  </div> 


## Halo lightcones 

We don't support reading halo lightcones using the toolkit yet so in this case you need to read them directly. However everything is in hdf5 format, with standard HACC units, so most of you should be reasonably comfortable with this anyway. If not feel free to ask. 

Let's start off just by reading the data. It should look something like 

```python
dataset = h5py.File('halolcproperties.hdf5','r')
print(dataset.keys())
fof_halo_mass = dataset['fof_halo_mass'][:]
fof_halo_center_x = dataset['fof_halo_x'][:]
```
and the same halo finder has been run on the lightcone so all the properties should look familiar. Some exceptions are the scale factor and the replication number. If you don't have the ra or declination, you can get this using healpy with 

```python
import healpy as hp
ra,dec = hp.vec2ang([x,y,z],lonlat=True)
```

Go ahead and try finding the most massive halo in this sample. Then let's open up the map! This is in healsparse format, but we can easily convert to a normal healpix format and visualize the CMB lensing map around the massive cluster. Let's give it a go, it will be something like..

```python
import healsparse as hsp 
import healpy as hp
import matplotlib.pyplot as plt

hsp_map = hsp.HealSparseMap.read('map.fits')
hp_map = hsp_map.generate_healpix_map(nside=2048)
dx=1.0
hp.cartview(hp_map,lonra=[ra-dx,ra+dx], latra=[dec-dx,dec+dx], return_projected_map=True)
plt.show()

```

## Challenges

- Make a kappa profile around the massive cluster, and compare to the expected profile using the parameters from the halo properties in the halo lightcone catalog. Can you recover the signal?
- Make a comoving distance to redshift plot of the halo lightcones (this might seem silly but we've caught code bugs this way many times before)
- Can you find this most massive halo in the snapshot data? Does it look any different? Note that the fof halo tag is *not* constant between the lightcone and the snapshot, you'll need to find it by matching on proximity in position and mass. 